In [ ]:
from google.colab import drive
drive.mount('/content/drive')

data_root='/content/drive/MyDrive/sampah/'



Mounted at /content/drive


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']

data_file = open(data_root + '/sampah.json').read()
intents = json.loads(data_file)

In [ ]:
intents

{'intents': [{'tag': 'dialog.introduction',
   'patterns': ['hai, perkenalkan aku adalah %name%',
    'hai nama aku adalah %name%',
    'perkenalkan nama aku %name%',
    'lo siapa'],
   'responses': ['Hai, aku adalah greenBot.',
    'Hai aku greenBot, Salam kenal.',
    'perkenalkan aku GreenBot, senang mengenal kamu :)',
    'Hallo, aku GreenBot, apa ada yang bisa saya bantu?']},
  {'tag': 'greetings.islam',
   'patterns': ['assalamualaikum',
    'assalamualaikum wr.wb',
    'Assalamualaikum',
    'Assalamualaikum wr.wb'],
   'responses': ['Waalaikumsalam! Selamat datang! Bagaimana aku bisa membantu Anda hari ini Jika Anda memiliki pertanyaan atau topik tertentu yang ingin dibahas, silakan beri tahu aku.',
    'Waalaikumsalam wr.wb! Selamat datang! Bagaimana aku bisa membantu Anda hari ini Jika Anda memiliki pertanyaan atau topik tertentu yang ingin dibahas, silakan beri tahu aku.',
    'Waalaikumsalam! Selamat datang! Bagaimana aku bisa membantu Anda hari ini Jika Anda memiliki pert

In [ ]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [36]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))


440 documents
108 classes ['dialog.3R', 'dialog.BahayaMenumpukSampah', 'dialog.BankSampah', 'dialog.BeritaSampah', 'dialog.BerpartisipasiProgramBankSampah', 'dialog.CaraKerjaAplikasi', 'dialog.ContohKemasanRamahLingkungan', 'dialog.ContohProdukRamahLingkungan', 'dialog.ContohSampahRumahtangga', 'dialog.EdukasiSampah', 'dialog.FasilitasDaurUlang', 'dialog.FiturMobile', 'dialog.FiturWeb', 'dialog.HargaSampah', 'dialog.IdeBisnisSampah', 'dialog.JamOperasionalBankSampah', 'dialog.JangkauanAplikasi', 'dialog.KebijakanPemerintah', 'dialog.KemasanRamahLingkungan', 'dialog.KerajinanSampah', 'dialog.KomunitasPeduliLingkungan', 'dialog.KondisiSampahDiIndonesia', 'dialog.LamaWaktuSampahNonOrganikTerurai', 'dialog.LamaWaktuSampahOrganikTerurai', 'dialog.LamaWaktuSampahTerurai', 'dialog.MengajakMasyarakat', 'dialog.MengatasiSampahBerbahaya', 'dialog.MetodePengolahanSampah', 'dialog.PanduanKompos', 'dialog.PenangananSampahNegaraMaju', 'dialog.PengangkutanSampah', 'dialog.PengelolaanSampahPerkotaanDa

In [ ]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


<ipython-input-29-1a891a7a8859>:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [ ]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

import tensorflow as tf

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = tf.keras.optimizers.legacy.SGD (lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
88/88 [==============================] - 1s 3ms/step - loss: 4.6866 - accuracy: 0.0068
Epoch 2/200
88/88 [==============================] - 0s 3ms/step - loss: 4.6392 - accuracy: 0.0364
Epoch 3/200
88/88 [==============================] - 0s 4ms/step - loss: 4.5618 - accuracy: 0.0455
Epoch 4/200
88/88 [==============================] - 0s 3ms/step - loss: 4.4538 - accuracy: 0.0614
Epoch 5/200
88/88 [==============================] - 0s 4ms/step - loss: 4.3047 - accuracy: 0.0864
Epoch 6/200
88/88 [==============================] - 0s 3ms/step - loss: 4.1255 - accuracy: 0.1068
Epoch 7/200
88/88 [==============================] - 0s 4ms/step - loss: 3.8797 - accuracy: 0.1227
Epoch 8/200
88/88 [==============================] - 0s 3ms/step - loss: 3.7139 - accuracy: 0.1273
Epoch 9/200
88/88 [==============================] - 0s 3ms/step - loss: 3.3821 - accuracy: 0.2091
Epoch 10/200
88/88 [==============================] - 0s 3ms/step - loss: 3.1747 - accuracy: 0.2250
Epoch 11/

In [37]:
from tensorflow.keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random

intents = json.loads(open(data_root + '/sampah.json').read())

words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [38]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [48]:
import ipywidgets as widgets
from IPython.display import display

def send_message(sender):
    msg = input_box.value
    input_box.value = ''

    if msg != '':
        chat_log.append('You: ' + msg)
        res = chatbot_response(msg)
        chat_log.append('Bot: ' + res)

        # Display the updated chat log
        output_area.clear_output(wait=True)
        with output_area:
            for line in chat_log:
                print(line)

# Create UI elements
input_box = widgets.Textarea(value='', placeholder='Type here...', layout={'height': '50px', 'width': '300px'})
send_button = widgets.Button(description='Send', layout={'width': '80px'})
output_area = widgets.Output(layout={'height': '300px'})

# Bind the function to the button click event
send_button.on_click(send_message)

# Display UI
display(widgets.VBox([input_box, send_button, output_area]))

# Initialize chat log
chat_log = []


1/1 [==============================] - 0s 20ms/step
